# Prompt engineering

## Goal

First steps with prompt engineering:

- Choose a validation dataset
- Implement a validation pipeline
- Try different prompt strategies

## Imports

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
import gc
import time
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from tqdm.auto import tqdm
import yaml
import os
import hashlib

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Registers the ops.

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

pd.set_option('display.max_colwidth', 200) 

## Code

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
    llm_int8_enable_fp32_cpu_offload= True)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
auto_device_map = {
    'model.embed_tokens': 0,
    'model.layers.0': 0,
    'model.layers.1': 0,
    'model.layers.2': 0,
    'model.layers.3': 0,
    'model.layers.4': 0,
    'model.layers.5': 0,
    'model.layers.6': 0,
    'model.layers.7': 0,
    'model.layers.8': 0,
    'model.layers.9': 0,
    'model.layers.10': 0,
    'model.layers.11': 0,
    'model.layers.12': 0,
    'model.layers.13': 0,
    'model.layers.14': 1,
    'model.layers.15': 1,
    'model.layers.16': 1,
    'model.layers.17': 1,
    'model.layers.18': 1,
    'model.layers.19': 1,
    'model.layers.20': 1,
    'model.layers.21': 1,
    'model.layers.22': 1,
    'model.layers.23': 1,
    'model.layers.24': 1,
    'model.layers.25': 1,
    'model.layers.26': 1,
    'model.layers.27': 1,
    'model.layers.28': 1,
    'model.layers.29': 1,
    'model.layers.30': 1,
    'model.layers.31': 1,
    'model.norm': 1,
    'lm_head': 1
 }

def create_shared_device_map(transition_layer):
    shared_device_map = {}
    for idx, key in enumerate(auto_device_map):
        if idx <= transition_layer:
            shared_device_map[key] = 0
        else:
            shared_device_map[key] = 1
    return shared_device_map

def create_intertwined_device_map():
    device_map = {}
    for idx, key in enumerate(auto_device_map):
        if idx == 0:
            device_map[key] = 1
        elif idx >= 33:
            device_map[key] = 0
        else:
            device_map[key] = idx % 2
    return device_map

In [ ]:
model_path = '/mnt/hdd0/Kaggle/llm_prompt_recovery/models/mixtral-8x7b-instruct-v0.1-hf'
model_path = '/home/gbarbadillo/data/mixtral-8x7b-instruct-v0.1-hf/'
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    #device_map="auto",
    device_map=create_intertwined_device_map(),
    #device_map=create_shared_device_map(16),
    trust_remote_code=True,
    #attn_implementation="flash_attention_2",
    )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id # this is needed to do batch inference
gc.collect()

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

def chat_with_mixtral(prompt, max_new_tokens=200, verbose=True, do_sample=False, temperature=0.7, top_p=0.95):
    if not prompt.startswith('<s>[INST]'):
        print('Formatting the prompt to Mixtral needs.')
        prompt = f'<s>[INST] {prompt} [/INST]'
    start = time.time()

    if do_sample:
        sampling_kwargs = dict(do_sample=True, temperature=temperature, top_p=top_p)
    else:
        sampling_kwargs = dict(do_sample=False)

    sequences = pipe(
        prompt ,
        max_new_tokens=max_new_tokens,
        # https://www.reddit.com/r/LocalLLaMA/comments/184g120/mistral_fine_tuning_eos_and_padding/
        # https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1/discussions/106
        pad_token_id=tokenizer.eos_token_id,
        **sampling_kwargs,
        return_full_text=False,
    )
    response = sequences[0]['generated_text']
    response = re.sub(r'[\'"]', '', response)
    if verbose:
        stop = time.time()
        time_taken = stop-start
        n_tokens = len(tokenizer.tokenize(response))
        print(f"Execution Time : {time_taken:.1f} s, tokens per second: {n_tokens/time_taken:.1f}")
    return response

In [ ]:
def print_gpu_memory():
    for device in range(torch.cuda.device_count()):
        print(f'GPU {device} memory allocated: {torch.cuda.memory_allocated(device)/1024**3:.1f} GB, max memory allocated: {torch.cuda.max_memory_allocated(device)/1024**3:.1f} GB')

In [ ]:
raise

## Flash attention speedup

https://huggingface.co/docs/transformers/en/model_doc/mixtral#speeding-up-mixtral-by-using-flash-attention

```bash
export FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE  
pip install -U flash-attn --no-build-isolation
```

In [ ]:
for _ in range(2):
    print(chat_with_mixtral('write a poem about real madrid', max_new_tokens=50))
print_gpu_memory()

In [ ]:
with open('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/bible.txt', 'r') as f:
    bible = f.read()
print(chat_with_mixtral(bible[:24000], max_new_tokens=50))

```
# flash-attention
Execution Time : 5.4 s, tokens per second: 9.4
Execution Time : 9.1 s, tokens per second: 5.6

# baseline
Execution Time : 5.1 s, tokens per second: 10.0
Execution Time : 9.2 s, tokens per second: 5.5
```

Not clear difference.

## Verify that batch prediction works

In [ ]:
prompts = [f'<s>[INST] What is the history of {country}? [/INST]' for country in ['Spain', 'France', 'Germany', 'Italy']]
pipe(prompts, do_sample=False, return_full_text=False, pad_token_id=tokenizer.eos_token_id, max_new_tokens=50)

In [ ]:
[chat_with_mixtral(prompt, max_new_tokens=50) for prompt in prompts]

In [ ]:
batch_size = 4
pipe_bs = pipeline(task="text-generation", model=model, tokenizer=tokenizer, batch_size=batch_size)
pipe_bs(prompts, do_sample=False, return_full_text=False, pad_token_id=tokenizer.eos_token_id, max_new_tokens=50)

In [ ]:
pipe_bs(prompts[:3] + [bible[:24000]], do_sample=False, return_full_text=False, pad_token_id=tokenizer.eos_token_id, max_new_tokens=50)

It seems the duration when batching inputs is determined by the longest input and output. Thus it might have sense to order the inputs by length.

All the predictions are the same, despite the method used.

## Study public datasets

https://www.kaggle.com/competitions/llm-prompt-recovery/discussion/481811

How can I study the datasets? I have to read them

I could read the prompts, this needs time.

## Dataset cleaning

In [ ]:
def curate_dataset(filepath):
    """ Search for sure and rewritten on the text to see the patterns """
    df = pd.read_csv(filepath)
    remove_texts = [
        "Sure, here's the rewritten text:",
        "**Rewritten Text:**",
        "**Rewritten text:**",
        "Certainly. Here's the rewritten text:",
    ]
    for text in remove_texts:
        df['rewritten_text'] = df['rewritten_text'].apply(lambda x: x.replace(text, ""))

    patterns = [
        r"^Sure.*?:"
    ]
    for pattern in patterns:
        df['rewritten_text'] = df['rewritten_text'].apply(lambda x: re.sub(pattern, "", x))
    df.to_csv(filepath.replace('.csv', '_curated.csv'), index=False)

curate_dataset('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite.csv')

## Evaluation

### First steps

In [ ]:
test = pd.read_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/nbroad-v1.csv').head(64)
test.head()

In [ ]:
MODEL_PROMPT = """<s>[INST]
# Guess the prompt

Your task is to guess which prompt was given to transform the original text into the rewritten text.
Read the two texts carefully and answer with the most likely prompt that was used to modify the original text.

- The response should be as short and concise as possible. Whenever possible use a single short sentence prompt.
- The prompt should be as generic as possible and content agnostic unless it is absolutely necessary to make reference to the content of the original text

## Original text

{original_text}

## Rewritten text

{rewritten_text}

[/INST]

## Prompt

"""

def truncate_txt(text, max_words=2000):
    text_list = text.split()
    return " ".join(text_list[:max_words])

prompts = []
for _, row in test.iterrows():
    prompts.append(MODEL_PROMPT.format(original_text=truncate_txt(row['original_text']),
                                       rewritten_text=truncate_txt(row['rewritten_text'])))

plt.hist([len(tokenizer.tokenize(prompt)) for prompt in prompts], bins=20);
plt.title('Distribution of prompt token length')
plt.xlabel('Prompt token length');

In [ ]:
responses = [chat_with_mixtral(prompt, max_new_tokens=25, verbose=False) for prompt in tqdm(prompts, smoothing=0)]

Making the predictions one by one takes 3m2s

In [ ]:
batch_size = 8
responses_bs = pipe(prompts, do_sample=False, return_full_text=False,
                    pad_token_id=tokenizer.eos_token_id, max_new_tokens=25,
                    batch_size=batch_size)

In [ ]:
prompt_lengths = [len(tokenizer.tokenize(prompt)) for prompt in prompts]
sorted_prompts = [prompt for _, prompt in sorted(zip(prompt_lengths, prompts))]

In [ ]:
batch_size = 8
responses_bs = pipe(sorted_prompts, do_sample=False, return_full_text=False,
                    pad_token_id=tokenizer.eos_token_id, max_new_tokens=25,
                    batch_size=batch_size)

In [ ]:
from torch.utils.data import Dataset
class PromptsDataset(Dataset):
    def __init__(self, prompts):
        self.prompts = prompts

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        return self.prompts[idx]

In [ ]:
batch_size = 8
responses_bs = pipe(PromptsDataset(sorted_prompts), do_sample=False, return_full_text=False,
                    pad_token_id=tokenizer.eos_token_id, max_new_tokens=25,
                    batch_size=batch_size)
responses_bs = list(responses_bs)

In [ ]:
batch_size = 16
responses_bs = pipe(sorted_prompts, do_sample=False, return_full_text=False,
                    pad_token_id=tokenizer.eos_token_id, max_new_tokens=25,
                    batch_size=batch_size)

In [ ]:
responses_bs

In [ ]:
print(MODEL_PROMPT)

- I cannot use a batch size higher than 8, which results in a very modest speedup of 2min vs 3min. And that is considering the current input tokens which may vary.
- The batch affects to some of the responses, it might be due to numerical errors
- Using a torch dataset does not speedup the inference

Thus I believe the best option is to use a simple sequential inference. To be fast I should use a small evaluation dataset.

### Some responses change when using batches

In [ ]:
idx = 4
batch_size = 1
print(pipe(prompts[idx: idx + batch_size], do_sample=False, return_full_text=False, pad_token_id=tokenizer.eos_token_id, max_new_tokens=25, batch_size=batch_size)[0][0]['generated_text'])

batch_size = 4
print(pipe(prompts[idx: idx + batch_size], do_sample=False, return_full_text=False, pad_token_id=tokenizer.eos_token_id, max_new_tokens=25, batch_size=batch_size)[0][0]['generated_text'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    padding_side="left",
    padding='do_not_pad')
tokenizer.pad_token_id = tokenizer.eos_token_id # this is needed to do batch inference
tokenizer.pad_token = tokenizer.eos_token

gc.collect()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)


batch_size = 2
pipe(prompts[:batch_size], do_sample=False, return_full_text=False, pad_token_id=tokenizer.eos_token_id, max_new_tokens=25, batch_size=batch_size)

### Code

In [ ]:
hub_url = "https://www.kaggle.com/models/google/sentence-t5/frameworks/TensorFlow2/variations/st5-base/versions/1"
encoder = hub.KerasLayer(hub_url)

RESULTS_FILEPATH = '/mnt/hdd0/Kaggle/llm_prompt_recovery/evaluations/mixtral_prompt_engineering.yaml'


def evaluate_prompt_template(prompt_template, max_new_tokens, test_filepath, n_samples,
                             random_seed=7, verbose=False, filter_multiline_responses=True):
    if is_already_evaluated(prompt_template, test_filepath, random_seed, max_new_tokens, n_samples):
        print('Already evaluated')
        return
    test = load_test_data(test_filepath, n_samples, random_seed)
    prompts = create_prompts(test, prompt_template, verbose=verbose)
    responses = [chat_with_mixtral(prompt, max_new_tokens=max_new_tokens, verbose=False)
                 for prompt in tqdm(prompts, smoothing=0, desc='Inference')]
    if filter_multiline_responses:
        responses = [response.split('\n')[0] for response in responses]
    test = evaluate_responses(test, responses, verbose=verbose)
    save_results(test, prompt_template, test_filepath, random_seed, max_new_tokens)
    return test

def load_test_data(test_filepath, n_samples, random_seed=7):
    test = pd.read_csv(test_filepath)
    np.random.seed(random_seed)
    keep_indices = np.random.choice(test.index, n_samples, replace=False)
    test = test.loc[keep_indices]
    return test

def truncate_txt(text, max_words=2000):
    text_list = text.split()
    return " ".join(text_list[:max_words])

def create_prompts(test, prompt_template, verbose=True):
    prompts = []
    for _, row in test.iterrows():
        prompts.append(prompt_template.format(original_text=truncate_txt(row['original_text']),
                                              rewritten_text=truncate_txt(row['rewritten_text'])))
    if verbose:
        plt.hist([len(tokenizer.tokenize(prompt)) for prompt in prompts], bins=20);
        plt.title('Distribution of prompt token length')
        plt.xlabel('Prompt token length');
        plt.show()
    return prompts

def evaluate_responses(test, responses, verbose=True):
    test['predicted_prompt'] = responses
    responses_embeddings = compute_t5_embeddings(responses)
    ground_truth_embeddings =compute_t5_embeddings(test['rewrite_prompt'])
    similarity = np.sum(responses_embeddings * ground_truth_embeddings, axis=1)
    test['cosine_similarity'] = similarity
    test['sharpened_cosine_similarity'] = similarity**3
    print(f'Mean sharpened cosine similarity: {test["sharpened_cosine_similarity"].mean():.3f} +- {test["sharpened_cosine_similarity"].std()/np.sqrt(len(test))*1.96:.3f}')
    if verbose:
        plt.hist(test['sharpened_cosine_similarity'], bins=20);
        plt.title('Distribution of sharpened cosine similarity');
        plt.show()
    return test

def compute_t5_embeddings(texts):
    """ I have verified that the embeddings have unit norm """
    embeddings = encoder(tf.constant(texts))[0].numpy()
    return embeddings

def save_results(test, prompt_template, test_filepath, random_seed,
                 max_new_tokens, output_filepath=RESULTS_FILEPATH):
    results = load_results(output_filepath)
    if not test_filepath in results:
        results[test_filepath] = dict()

    hash = get_evaluation_hash(prompt_template, test_filepath, random_seed, max_new_tokens, len(test))
    results[test_filepath][hash] = dict(
        prompt_template=prompt_template,
        max_new_tokens=max_new_tokens,
        random_seed=random_seed,
        n_samples=len(test),
        responses=test['predicted_prompt'].tolist(),
        ids=test['id'].tolist(),
        sharpened_cosine_similarity=test['sharpened_cosine_similarity'].tolist(),
        mean_sharpened_cosine_similarity=round(float(test['sharpened_cosine_similarity'].mean()), 4),
    )
    with open(output_filepath, 'w') as f:
        yaml.dump(results, f)

def load_results(output_filepath):
    if os.path.exists(output_filepath):
        with open(output_filepath, 'r') as f:
            results = yaml.load(f, Loader=yaml.FullLoader)
    else:
        results = {}
    return results


def is_already_evaluated(prompt_template, test_filepath, random_seed, max_new_tokens, n_samples, output_filepath=RESULTS_FILEPATH):
    hash = get_evaluation_hash(prompt_template, test_filepath, random_seed, max_new_tokens, n_samples)
    results = load_results(output_filepath)
    return hash in results.get(test_filepath, dict())


def get_evaluation_hash(prompt_template, test_filepath, random_seed, max_new_tokens, n_samples):
    input_string = f"{prompt_template}{test_filepath}{random_seed}{max_new_tokens}{n_samples}"
    return create_hash(input_string)

def create_hash(input_string, algorithm='sha256'):
    try:
        hash_object = hashlib.new(algorithm)
    except ValueError:
        raise ValueError(f"Unsupported hashing algorithm: {algorithm}")
    hash_object.update(input_string.encode())
    return hash_object.hexdigest()

In [ ]:
def show_worse_predictions(test, n=5):
    display_columns = ['original_text', 'rewritten_text', 'rewrite_prompt', 'predicted_prompt', 'sharpened_cosine_similarity']
    display_columns = ['rewrite_prompt', 'predicted_prompt', 'sharpened_cosine_similarity']
    return test[display_columns].sort_values('sharpened_cosine_similarity', ascending=True).head(n)

### Evaluations

I'm going to try to optimize the results on `'/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv'`

For few shot prompting I will be using samples from:

- ``

In [ ]:
prompt_template = """<s>[INST]
# Guess the instruction

Your task is to guess which instruction was given to transform the original text into the rewritten text.
Read the two texts carefully and answer with the most likely prompt that was used to modify the original text.

- The response should be as short and concise as possible. Whenever possible use a single short sentence instruction.
- The instruction should be as generic as possible and content agnostic unless it is absolutely necessary to make reference to the content of the original text

## Original text

{original_text}

## Rewritten text

{rewritten_text}

[/INST]

## Instruction

"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
display_columns = ['original_text', 'rewritten_text', 'rewrite_prompt', 'predicted_prompt', 'sharpened_cosine_similarity']
display_columns = ['rewrite_prompt', 'predicted_prompt', 'sharpened_cosine_similarity']
test[display_columns].sort_values('sharpened_cosine_similarity', ascending=True).head()

In [ ]:
forum_prompt = """<s>[INST] Original Text: Hey there! Just a heads up: our friendly dog may bark a bit, but don't worry, he's all bark and no bite![/INST]
Provide the modified text and I'll tell you a something general that changed about it.  I'll avoid any specifics though.  My tone will be neutral.</s>
[INST] Re-written Text: Warning: Protective dog on premises. May exhibit aggressive behavior. Ensure personal safety by maintaining distance and avoiding direct contact.[/INST]
The request was: Shift the narrative from offering an informal, comforting assurance to providing a more structured advisory note that emphasizes mindfulness and preparedness. This modification should mark a clear change in tone and purpose, aiming to responsibly inform and guide, while maintaining a positive and constructive approach.</s>
<s>[INST] Original Text: A lunar eclipse happens when Earth casts its shadow on the moon during a full moon. The moon appears reddish because Earth's atmosphere scatters sunlight, some of which refracts onto the moon's surface. Total eclipses see the moon entirely in Earth's shadow; partial ones occur when only part of the moon is shadowed.[/INST]
Provide the modified text and I'll tell you a something general that changed about it.  I'll avoid any specifics though.  My tone will be neutral.</s>
[INST] Re-written Text: Yo check it, when the Earth steps in, takes its place, casting shadows on the moon's face. It's a full moon night, the scene's set right, for a lunar eclipse, a celestial sight. The moon turns red, ain't no dread, it's just Earth's atmosphere playing with sunlight's thread, scattering colors, bending light, onto the moon's surface, making the night bright. Total eclipse, the moon's fully in the dark, covered by Earth's shadow, making its mark. But when it's partial, not all is shadowed, just a piece of the moon, slightly furrowed. So that's the rap, the lunar eclipse track, a dance of shadows, with no slack. Earth, moon, and sun, in a cosmic play, creating the spectacle we see today.[/INST]
The request was: Transform your communication from an academic delivery to a dynamic, rhythm-infused presentation. Keep the essence of the information intact but weave in artistic elements, utilizing rhythm, rhyme, and a conversational style. This approach should make the content more relatable and enjoyable, aiming to both educate and entertain your audience.</s>
<s>[INST] Original Text: {original_text} [/INST]
Provide the modified text and I'll tell you a something general that changed about it.  I'll avoid any specifics though.  My tone will be neutral.</s>
[INST] Re-written Text: {rewritten_text} [/INST]
The request was: """
test = evaluate_prompt_template(
    forum_prompt, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
display_columns = ['original_text', 'rewritten_text', 'rewrite_prompt', 'predicted_prompt', 'sharpened_cosine_similarity']
display_columns = ['rewrite_prompt', 'predicted_prompt', 'sharpened_cosine_similarity']
test[display_columns].sort_values('sharpened_cosine_similarity', ascending=True).head()

Let's try with few shot prompts. Let's give good examples to the model.

In [ ]:
prompt_template = """<s>[INST]
Analyze the original and rewritten text and answer with the most likely instruction that was given
to rewrite or make stylistic changes to the original text.

## Original text

{original_text}

## Rewritten text

{rewritten_text}

[/INST]

## Instruction


"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
prompt_template = """<s>[INST]
Analyze the original and rewritten text and answer with the most likely instruction that was given
to rewrite or make stylistic changes to the original text.

Below you can see some examples and finally the task you have to do.

## Example 1

### Original text

The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

### Rewritten text

(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be

### Instruction

Convert this into a sea shanty

## Example 2

### Original text

Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.

### Rewritten text

Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.

### Instruction

Rewrite this essay but do it using the writing style of William Shakespeare

## Task

### Original text

{original_text}

### Rewritten text

{rewritten_text}

[/INST]

### Instruction


"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
prompt_template = """<s>[INST]
Analyze the original and rewritten text and answer with the most likely instruction that was given
to rewrite or make stylistic changes to the original text.

Below you can see some examples and finally the task you have to do.

## Example 1

### Original text

The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

### Rewritten text

(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be

### Instruction

Convert this into a sea shanty

## Example 2

### Original text

Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.

### Rewritten text

Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.

### Instruction

Rewrite this essay but do it using the writing style of William Shakespeare

## Example 3

### Original text

In a small town nestled between rolling hills and vast fields, there lived a cat named Oliver. Every evening, as the sun dipped below the horizon, painting the sky in shades of orange and purple, Oliver would embark on his nightly adventures. With the moon as his guide, he explored hidden nooks, greeted nocturnal friends, and chased fleeting shadows. His life was simple yet full of wonder, a testament to finding joy in the smallest of moments.

### Rewritten text

En un pequeño pueblo encajado entre colinas ondulantes y vastos campos, vivía un gato llamado Oliver. Cada atardecer, cuando el sol se sumergía bajo el horizonte, pintando el cielo de tonos naranjas y morados, Oliver emprendía sus aventuras nocturnas. Con la luna como su guía, exploraba rincones ocultos, saludaba a amigos nocturnos y perseguía sombras fugaces. Su vida era sencilla pero llena de maravillas, un testimonio de encontrar alegría en los momentos más pequeños.

### Instruction

Translate to spanish

## Task

### Original text

{original_text}

### Rewritten text

{rewritten_text}

[/INST]

### Instruction


"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
prompt_template = """<s>[INST]
Analyze the original and rewritten text and answer with the most likely instruction that was given
to rewrite or make stylistic changes to the original text.

Below you can see some examples and finally the task you have to do.

## Example 1

### Original text

The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

### Rewritten text

(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be

### Instruction

Convert this into a sea shanty

## Example 2

### Original text

Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.

### Rewritten text

Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.

### Instruction

Rewrite this essay but do it using the writing style of William Shakespeare

## Example 3

### Original text

In a small town nestled between rolling hills and vast fields, there lived a cat named Oliver. Every evening, as the sun dipped below the horizon, painting the sky in shades of orange and purple, Oliver would embark on his nightly adventures. With the moon as his guide, he explored hidden nooks, greeted nocturnal friends, and chased fleeting shadows. His life was simple yet full of wonder, a testament to finding joy in the smallest of moments.

### Rewritten text

En un pequeño pueblo encajado entre colinas ondulantes y vastos campos, vivía un gato llamado Oliver. Cada atardecer, cuando el sol se sumergía bajo el horizonte, pintando el cielo de tonos naranjas y morados, Oliver emprendía sus aventuras nocturnas. Con la luna como su guía, exploraba rincones ocultos, saludaba a amigos nocturnos y perseguía sombras fugaces. Su vida era sencilla pero llena de maravillas, un testimonio de encontrar alegría en los momentos más pequeños.

### Instruction

Translate to spanish

## Task

### Original text

{original_text}

### Rewritten text

{rewritten_text}

[/INST]

### Instruction


"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
prompt_template = """<s>[INST]
Analyze the original and rewritten text and answer with the most likely instruction that was given
to rewrite or make stylistic changes to the original text.

Below you can see some examples and finally the task you have to do.

## Example 1

### Original text

The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

### Rewritten text

(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be

### Instruction

Convert this into a sea shanty

## Example 2

### Original text

Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.

### Rewritten text

Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.

### Instruction

Rewrite this essay but do it using the writing style of William Shakespeare

## Example 3

### Original text

In a small town nestled between rolling hills and vast fields, there lived a cat named Oliver. Every evening, as the sun dipped below the horizon, painting the sky in shades of orange and purple, Oliver would embark on his nightly adventures. With the moon as his guide, he explored hidden nooks, greeted nocturnal friends, and chased fleeting shadows. His life was simple yet full of wonder, a testament to finding joy in the smallest of moments.

### Rewritten text

En un pequeño pueblo encajado entre colinas ondulantes y vastos campos, vivía un gato llamado Oliver. Cada atardecer, cuando el sol se sumergía bajo el horizonte, pintando el cielo de tonos naranjas y morados, Oliver emprendía sus aventuras nocturnas. Con la luna como su guía, exploraba rincones ocultos, saludaba a amigos nocturnos y perseguía sombras fugaces. Su vida era sencilla pero llena de maravillas, un testimonio de encontrar alegría en los momentos más pequeños.

### Instruction

Translate to spanish

## Example 4

### Original text

Quantum mechanics is a fundamental theory in physics that describes the behavior of particles at the microscopic scale. It introduces concepts like wave-particle duality, quantization of energy, and uncertainty principles, fundamentally changing our understanding of matter and energy.

### Rewritten text

Quantum mechanics is like a magic rule book for tiny things like atoms and particles. It tells us that these tiny things can be in different places at the same time and can act like both balls and waves. It's how the smallest parts of our world work!

### Instruction

Improve the text as if you were explaining it to a five-year-old.

## Task

### Original text

{original_text}

### Rewritten text

{rewritten_text}

[/INST]

### Instruction


"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
prompt_template = """<s>[INST]
Analyze the original and rewritten text and answer with the most likely instruction that was given
to rewrite or make stylistic changes to the original text.

Below you can see some examples and finally the task you have to do.

## Example 1

### Original text

The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.

### Rewritten text

(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be

### Instruction

Convert this into a sea shanty

## Example 2

### Original text

Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.

### Rewritten text

Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.

### Instruction

Rewrite this essay but do it using the writing style of William Shakespeare

## Example 3

### Original text

In a small town nestled between rolling hills and vast fields, there lived a cat named Oliver. Every evening, as the sun dipped below the horizon, painting the sky in shades of orange and purple, Oliver would embark on his nightly adventures. With the moon as his guide, he explored hidden nooks, greeted nocturnal friends, and chased fleeting shadows. His life was simple yet full of wonder, a testament to finding joy in the smallest of moments.

### Rewritten text

En un pequeño pueblo encajado entre colinas ondulantes y vastos campos, vivía un gato llamado Oliver. Cada atardecer, cuando el sol se sumergía bajo el horizonte, pintando el cielo de tonos naranjas y morados, Oliver emprendía sus aventuras nocturnas. Con la luna como su guía, exploraba rincones ocultos, saludaba a amigos nocturnos y perseguía sombras fugaces. Su vida era sencilla pero llena de maravillas, un testimonio de encontrar alegría en los momentos más pequeños.

### Instruction

Translate to spanish

## Example 4

### Original text

Quantum mechanics is a fundamental theory in physics that describes the behavior of particles at the microscopic scale. It introduces concepts like wave-particle duality, quantization of energy, and uncertainty principles, fundamentally changing our understanding of matter and energy.

### Rewritten text

Quantum mechanics is like a magic rule book for tiny things like atoms and particles. It tells us that these tiny things can be in different places at the same time and can act like both balls and waves. It's how the smallest parts of our world work!

### Instruction

Improve the text as if you were explaining it to a five-year-old.

## Example 5

### Original text

In a quaint village, a forgotten ice cream recipe rekindled friendships long cooled. The shared delight of creamy sweetness under the sun's gaze melted barriers, spoonful by spoonful. Laughter echoed, as flavors of joy united the community, proving happiness is best served cold and shared.

### Rewritten text

A forgotten ice cream recipe brings a divided village together, rekindling friendships and joy.

### Instruction

Summarize the story in a single short sentence.

## Example 6

### Original text

In a quaint village, a forgotten ice cream recipe rekindled friendships long cooled. The shared delight of creamy sweetness under the sun's gaze melted barriers, spoonful by spoonful. Laughter echoed, as flavors of joy united the community, proving happiness is best served cold and shared.

### Rewritten text

Sweet Unity

### Instruction

What is the title of the story?

## Task

### Original text

{original_text}

### Rewritten text

{rewritten_text}

[/INST]

### Instruction


"""
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
len(tokenizer.tokenize(prompt_template))

In [ ]:
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/nbroad-v2.csv', n_samples=32)

In [ ]:
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/nbroad-v1.csv', n_samples=32)

In [ ]:
def format_messages_for_mixtral(messages, change_line_between_messages=True):
    text = '<s>'
    for message in messages:
        if message['role'] == 'user':
            text += f'[INST] {message["content"]} [/INST]'
        else:
            text += f'{message["content"]}</s>'
        if change_line_between_messages:
            text += '\n'
    return text

In [ ]:
intro = "Analyze the original and rewritten text and answer with the most likely instruction that was given to rewrite or make stylistic changes to the original text."

examples = [
    {
        'original_text': "The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.",
        'rewritten_text': "(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be",
        'instruction': "Convert this into a sea shanty"
    },
    {
        'original_text': "Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.",
        'rewritten_text': "Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.",
        'instruction': "Rewrite this essay but do it using the writing style of William Shakespeare"
    },
    {
        'original_text': "In a small town nestled between rolling hills and vast fields, there lived a cat named Oliver. Every evening, as the sun dipped below the horizon, painting the sky in shades of orange and purple, Oliver would embark on his nightly adventures. With the moon as his guide, he explored hidden nooks, greeted nocturnal friends, and chased fleeting shadows. His life was simple yet full of wonder, a testament to finding joy in the smallest of moments.",
        'rewritten_text': "En un pequeño pueblo encajado entre colinas ondulantes y vastos campos, vivía un gato llamado Oliver. Cada atardecer, cuando el sol se sumergía bajo el horizonte, pintando el cielo de tonos naranjas y morados, Oliver emprendía sus aventuras nocturnas. Con la luna como su guía, exploraba rincones ocultos, saludaba a amigos nocturnos y perseguía sombras fugaces. Su vida era sencilla pero llena de maravillas, un testimonio de encontrar alegría en los momentos más pequeños.",
        'instruction': "Translate to spanish"
    },
    {
        'original_text': "Quantum mechanics is a fundamental theory in physics that describes the behavior of particles at the microscopic scale. It introduces concepts like wave-particle duality, quantization of energy, and uncertainty principles, fundamentally changing our understanding of matter and energy.",
        'rewritten_text': "Quantum mechanics is like a magic rule book for tiny things like atoms and particles. It tells us that these tiny things can be in different places at the same time and can act like both balls and waves. It's how the smallest parts of our world work!",
        'instruction': "Improve the text as if you were explaining it to a five-year-old"
    },
    {
        'original_text': "In a quaint village, a forgotten ice cream recipe rekindled friendships long cooled. The shared delight of creamy sweetness under the sun's gaze melted barriers, spoonful by spoonful. Laughter echoed, as flavors of joy united the community, proving happiness is best served cold and shared.",
        'rewritten_text': "A forgotten ice cream recipe brings a divided village together, rekindling friendships and joy.",
        'instruction': "Summarize the story in a single short sentence"
    },
    {
        'original_text': "In a quaint village, a forgotten ice cream recipe rekindled friendships long cooled. The shared delight of creamy sweetness under the sun's gaze melted barriers, spoonful by spoonful. Laughter echoed, as flavors of joy united the community, proving happiness is best served cold and shared.",
        'rewritten_text': "Sweet Unity",
        'instruction': "What is the title of the story?"
    }
]

example_template = """
### Original text

{original_text}

### Rewritten text

{rewritten_text}

"""

messages = [
    {'role': 'user', 'content': intro},
    {'role': 'assistant', 'content': "I understand the task, I'm going to take a deep breath, concentrate and we can start."},
]
for example in examples:
    messages.append({'role': 'user', 'content': example_template.format(
        original_text=example['original_text'], rewritten_text=example['rewritten_text'])})
    messages.append({'role': 'assistant', 'content': example['instruction']})
messages.append({'role': 'user', 'content': example_template})
prompt_template = format_messages_for_mixtral(messages)
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
intro = "Analyze the original and rewritten text and answer with the most likely instruction that was given to rewrite or make stylistic changes to the original text."

examples = [
    {
        'original_text': "The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.",
        'rewritten_text': "(Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be",
        'instruction': "Convert this into a sea shanty"
    },
    {
        'original_text': "Chocolate, a beloved treat worldwide, originates from the cacao tree's seeds, native to the Americas. Its history spans over 3,000 years, evolving from a bitter beverage to the sweet delicacies we relish today. Esteemed by ancient civilizations like the Maya and Aztecs, chocolate was more than a food; it symbolized wealth, power, and even had spiritual significance. The Spanish introduction of chocolate to Europe transformed it into a global phenomenon, leading to the creation of diverse chocolate products. Today, chocolate is celebrated for its rich flavors and mood-enhancing properties, making it a universal symbol of indulgence and affection.",
        'rewritten_text': "Oh chocolate, thou sweet regale of tongues, birthed from the seeds of cacao's embrace, cradled in the Americas' bosom. Thine ancient roots weave back yon three millennia, transmuting from bitter draught to sweet ambrosia cherished in modern banquet. Held in high esteem by Maya and Aztec alike, thee was more than mere sustenance; a symbol of opulence, power, and the divine. When Spaniards didst bear thee across the seas, Europe's palate was forever changed, birthing a myriad of confections. Now, celebrated for thy velvety richness and spirit-lifting virtues, chocolate remains a beacon of delight and affection, universally adored.",
        'instruction': "Rewrite this essay but do it using the writing style of William Shakespeare"
    },
    {
        'original_text': "In a small town nestled between rolling hills and vast fields, there lived a cat named Oliver. Every evening, as the sun dipped below the horizon, painting the sky in shades of orange and purple, Oliver would embark on his nightly adventures. With the moon as his guide, he explored hidden nooks, greeted nocturnal friends, and chased fleeting shadows. His life was simple yet full of wonder, a testament to finding joy in the smallest of moments.",
        'rewritten_text': "En un pequeño pueblo encajado entre colinas ondulantes y vastos campos, vivía un gato llamado Oliver. Cada atardecer, cuando el sol se sumergía bajo el horizonte, pintando el cielo de tonos naranjas y morados, Oliver emprendía sus aventuras nocturnas. Con la luna como su guía, exploraba rincones ocultos, saludaba a amigos nocturnos y perseguía sombras fugaces. Su vida era sencilla pero llena de maravillas, un testimonio de encontrar alegría en los momentos más pequeños.",
        'instruction': "Translate to spanish"
    },
    {
        'original_text': "Quantum mechanics is a fundamental theory in physics that describes the behavior of particles at the microscopic scale. It introduces concepts like wave-particle duality, quantization of energy, and uncertainty principles, fundamentally changing our understanding of matter and energy.",
        'rewritten_text': "Quantum mechanics is like a magic rule book for tiny things like atoms and particles. It tells us that these tiny things can be in different places at the same time and can act like both balls and waves. It's how the smallest parts of our world work!",
        'instruction': "Improve the text as if you were explaining it to a five-year-old"
    },
    {
        'original_text': "In a quaint village, a forgotten ice cream recipe rekindled friendships long cooled. The shared delight of creamy sweetness under the sun's gaze melted barriers, spoonful by spoonful. Laughter echoed, as flavors of joy united the community, proving happiness is best served cold and shared.",
        'rewritten_text': "A forgotten ice cream recipe brings a divided village together, rekindling friendships and joy.",
        'instruction': "Summarize the story in a single short sentence"
    },
    {
        'original_text': "In a quaint village, a forgotten ice cream recipe rekindled friendships long cooled. The shared delight of creamy sweetness under the sun's gaze melted barriers, spoonful by spoonful. Laughter echoed, as flavors of joy united the community, proving happiness is best served cold and shared.",
        'rewritten_text': "Sweet Unity",
        'instruction': "What is the title of the story?"
    },
    {
        'original_text': "Hey! 😊 Hope you're doing awesome today. I was thinking, it’s been a while since we caught up, and what better way to do that than over some good food? How about dinner tonight? Let’s pick a place we both love or try out that new spot we've been talking about. What do you say? Let me know what works for you! 🍽️👯‍♂️",
        'rewritten_text': "I hope this message finds you well. I am reaching out to propose an opportunity for us to catch up, as it has been some time since our last meeting. Considering the importance of our friendship and our shared love for good cuisine, I suggest we convene for dinner this evening.\n\nThere is the option of visiting a restaurant we both hold in high regard or exploring a new establishment that has recently caught our interest. Please let me know your availability for tonight and if there is a specific venue you would prefer.\n\nI look forward to your prompt response and to the possibility of an enjoyable evening together.",
        'instruction': "Rewrite the text in a more formal tone."
    },
    {
        'original_text': '{original_text}',
        'rewritten_text': '{rewritten_text}',
        'instruction': '',
    }
]

example_template = """
### Original text

{original_text}

### Rewritten text

{rewritten_text}

"""

messages = [
    {'role': 'user', 'content': intro},
    {'role': 'assistant', 'content': "I understand the task, I'm going to take a deep breath, concentrate and we can start."},
]
for example in examples:
    messages.append({'role': 'user', 'content': f'Original text: {example["original_text"]}'})
    messages.append({'role': 'assistant', 'content': 'Give me the rewritten text and I will give you the instruction'})
    messages.append({'role': 'user', 'content': f'Rewritten text: {example["rewritten_text"]}'})
    messages.append({'role': 'assistant', 'content': f'The instruction was: {example["instruction"]}'})
prompt_template = format_messages_for_mixtral(messages, change_line_between_messages=False)[:-5]
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv', n_samples=32)

In [ ]:
show_worse_predictions(test, 10)

In [ ]:
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/nbroad-v2.csv', n_samples=32)

In [ ]:
test = evaluate_prompt_template(
    prompt_template, max_new_tokens=75,
    test_filepath='/mnt/hdd0/Kaggle/llm_prompt_recovery/data/nbroad-v1.csv', n_samples=32)

### Analyze results

## TODO

- [x] Evaluate with T5 transformer
- [x] Select the best dataset for testing
- [x] Estimate uncertainty on evaluation
- [x] Save evaluation results to disk for traceability
- [ ] Pairwise comparison to have more power to take decisions
- [x] Encapsulate the code to do a prompt evaluation
- [ ] Is it better to use a long instruction, or a conversation? Probably a conversation but verify it.